In [1]:
# Imports
load('prismatic_envelope.sage')
load('precision.py')
load('homology.sage')

### User-defined input

In [2]:
# The prime p
p=2

# The motivic weight i
i=16

# The power of the uniformizer n
n=2

# The residual degree f
# The present code only supports totally ramified extensions of Qp,
# i.e., where f=1
f=1

### Some precision calculations

In [3]:
# The calculated F-precision needed to compute at this weight
Fprec=n*i

# The target precision
target_precision=nygaard_exponent(p,i,n)

####################
# Precision losses #
####################

### From \delta
precision_loss_delta=math.floor(math.log(Fprec-1,p))

### From passing from OK to OK/pi^n
precision_loss_quotient=0
for q in range(p,i):
    precision_loss_quotient+=n*valuation(p,math.factorial(q))
    
### From lifting nabla to Nygaard
precision_loss_nygaard=n*math.floor(i*(i-1)/2)

### From computing can-phi on primitives
precision_loss_primitives=target_precision

### From renormalizing the Eisenstein polynomial
precision_loss_from_Eisenstein=1

total_precision=(target_precision+precision_loss_delta
                 +precision_loss_quotient
                 +precision_loss_nygaard
                 +precision_loss_primitives
                 +precision_loss_from_Eisenstein)

print("total p-adic precision is {}".format(total_precision))
print("Fprec is {}".format(Fprec))

# The coefficient ring W
if f==1:
    W=Zp(p,total_precision,type='capped-abs',print_mode='digits',show_prec=False)
else:
    W=Zq(p**f,total_precision,names='a',type='capped-abs',print_mode='series',show_prec=False)
    
# The Breuil-Kisin ring A
A.<z>=PowerSeriesRing(W,Fprec)

total p-adic precision is 309
Fprec is 32


### User-defined input: the Eisenstein polynomial

Note that in order for this to be created in a power series ring with the correct p-adic and F-adic precisions,
the elements p,i,n,f should be set above *before* defining the Eisenstein polynomial.

In [4]:
# The Eisenstein polynomial E
E=A(z+p)

### The main calculation

In [5]:
#%%capture
# Suppresses some Python warnings and SAGE variable injections

# The normalized Eisenstein polynomial
# The normalization is to bring the Eisenstein polynomial into the form E(0)=p
E=eisenstein_normalization(p,E)

# The syntomic matrices from new
%prun -s cumulative SyntomicComplex(p,i,n,E,total_precision,Fprec,debug=False)

         54044109 function calls (54014290 primitive calls) in 89.320 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000   89.320   89.320 {built-in method builtins.exec}
        1    0.000    0.000   89.320   89.320 <string>:14(__init__)
        1    0.056    0.056   89.303   89.303 <string>:787(syntomic_matrices)
        1    0.000    0.000   77.339   77.339 <string>:658(nablaP_matrix_OK)
      425    1.480    0.003   73.512    0.173 <string>:570(reduce)
       53    0.018    0.000   63.828    1.204 <string>:601(recursive_reduce)
   683241   53.500    0.000   58.522    0.000 multi_polynomial_element.py:317(_mul_)
        1    0.000    0.000   47.888   47.888 <string>:697(initialize_bk_factor)
   168354    0.135    0.000   18.038    0.000 multi_polynomial_element.py:352(_rmul_)
   168354   14.686    0.000   17.657    0.000 {method 'scalar_rmult' of 'sage.rings.polynomial.polydict.PolyDict' object

### Assembling the syntomic complex and computing its cohomology